In [1]:
# common packages
import pandas as pd
import time

# # DL framework
import torch
from torchtext import data

from attractivedata import AttractiveData
from trainer import AttractiveTrainer

## Load and prepare data

In [2]:
train_file = 'data/train.csv'
test_file = 'data/test.csv'
pretrained_file = 'glove.840B.300d'
config = {
    'max_size': 48,
    'min_freq': 5,
    'batch_size': 64,
    'pretrained_file': pretrained_file
}


In [3]:
AttractiveData = AttractiveData(train_file, test_file, pretrained_file, config)

In [4]:
for i, sentence in enumerate(AttractiveData.test_data):
    if i == 3:
        print(vars(AttractiveData.train_data[i]), vars(sentence))

{'Headline': ['Sorry', ',', 'i', 'spent', 'it', 'on', 'myself', '!', 'Harvey', 'Nichols', "'", 'hilarious', 'Christmas', 'advert', 'sees', 'people', 'treating', 'themselves', 'instead', 'of', 'others'], 'Category': 'femail', 'Label': '3.333333333333333'} {'Headline': ['Three', 'police', 'officers', 'accused', 'of', 'stealing', '?', '?', '30k', 'during', 'raid', 'on', 'criminal'], 'Category': 'news'}


In [5]:
len(AttractiveData.CATEGORIES_LABEL.vocab.freqs)

18

In [6]:
max_len = 0
a = AttractiveData.test_data
for i in range(len(a)):
    if len(a[i].Headline) >= max_len:
        max_len = len(a[i].Headline)
max_len

31

## Start to train

In [7]:
num_workers = 10

config['timestr'] = time.strftime("%Y%m%d-%H%M%S")
config['save_name'] = 'CNN_LSTM'
config['input_dim'] = len(AttractiveData.TEXT.vocab)
config['embedding_dim'] = 300
config['category_dim'] = len(AttractiveData.CATEGORIES_LABEL.vocab)
config['category_embedding_dim'] = 16
config['hidden_dim'] = 100
config['output_dim'] = 1
config['log_steps'] = 10
config['epochs'] = 100
config['lr'] = {
    'encoder': 1e-4,
    'embedding': 1e-5,
    'linear': 1e-4
}
config['num_layers'] = 2
config['nhead'] = 4
config['kernel_size'] = 2
config['dropout'] = 0.1

pretrained_embeddings = AttractiveData.TEXT.vocab.vectors
print(pretrained_embeddings.shape)

torch.Size([1518, 300])


In [8]:
# max(AttractiveData.df_train.Headline.str.len()), max(AttractiveData.df_test.Headline.str.len())

In [9]:
AttractiveTrainer = AttractiveTrainer(config, AttractiveData.device, AttractiveData.trainloader, pretrained_embeddings)

In [10]:
AttractiveTrainer.model, AttractiveTrainer.config['total_params'], AttractiveTrainer.config['total_learned_params']

(AttractiveNet(
   (embedding): AttractiveEmbedding(
     (token): TokenEmbedding(1518, 300, padding_idx=1)
     (position): PositionalEmbedding()
     (dropout): Dropout(p=0.1, inplace=False)
   )
   (category_embedding): CategoryEmbedding(18, 16, padding_idx=0)
   (cnn): Conv1d(300, 100, kernel_size=(2,), stride=(1,))
   (relu): ReLU()
   (encoder): LSTM(100, 100, num_layers=2, dropout=0.1, bidirectional=True)
   (linear_output): Linear(in_features=216, out_features=1, bias=True)
 ),
 919205,
 463805)

In [11]:
AttractiveTrainer.train()

Epoch:   1%|          | 1/100 [00:00<00:45,  2.16it/s]
EP_train | avg_loss: 7.534581363201141 |
Epoch:   2%|▏         | 2/100 [00:00<00:43,  2.23it/s]
EP_train | avg_loss: 0.5974896494299173 |
Epoch:   3%|▎         | 3/100 [00:01<00:42,  2.27it/s]
EP_train | avg_loss: 0.5571830421686172 |
Epoch:   4%|▍         | 4/100 [00:01<00:41,  2.31it/s]
EP_train | avg_loss: 0.552936103194952 |
Epoch:   5%|▌         | 5/100 [00:02<00:40,  2.33it/s]
EP_train | avg_loss: 0.5507670063525438 |
Epoch:   6%|▌         | 6/100 [00:02<00:39,  2.35it/s]
EP_train | avg_loss: 0.5503775095567107 |
Epoch:   7%|▋         | 7/100 [00:02<00:39,  2.37it/s]
EP_train | avg_loss: 0.5487625757232308 |
Epoch:   8%|▊         | 8/100 [00:03<00:38,  2.39it/s]
EP_train | avg_loss: 0.5479099610820413 |
Epoch:   9%|▉         | 9/100 [00:03<00:38,  2.39it/s]
EP_train | avg_loss: 0.5486676758155227 |
Epoch:  10%|█         | 10/100 [00:04<00:37,  2.39it/s]
EP_train | avg_loss: 0.5476385513320565 |
Epoch:  11%|█         | 11/100 

## for classification, not better

In [14]:
# from sklearn.metrics import mean_squared_error
# a = AttractiveTrainer.train_predict
# AttractiveData.LABEL.vocab.itos[int(a[0])], AttractiveTrainer.train_true[0]
# correct = 0
# pred_list = []
# true_list = []
# for i in range(len(a)):
#     pred = AttractiveData.LABEL.vocab.itos[int(a[i])]
#     pred_list.append(float(pred))
#     true = AttractiveData.LABEL.vocab.itos[int(AttractiveTrainer.train_true[i])]
#     true_list.append(float(true))
# mean_squared_error(true_list, pred_list)
# # true_list

0.5601443355119825

## Below is testing

In [12]:
from transformermodel import TransformerModel
from attractivenet import AttractiveNet
PATH = './model/CNN_LSTM_20201031-141425_0.5395.100'
# load_model = TransformerModel(config).to(AttractiveData.device)
load_model = AttractiveNet(config).to(AttractiveData.device)
load_model.load_state_dict(torch.load(PATH))
load_model.eval()

AttractiveNet(
  (embedding): AttractiveEmbedding(
    (token): TokenEmbedding(1518, 300, padding_idx=1)
    (position): PositionalEmbedding()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (category_embedding): CategoryEmbedding(18, 16, padding_idx=0)
  (cnn): Conv1d(300, 100, kernel_size=(2,), stride=(1,))
  (relu): ReLU()
  (encoder): LSTM(100, 100, num_layers=2, dropout=0.1, bidirectional=True)
  (linear_output): Linear(in_features=216, out_features=1, bias=True)
)

In [13]:
def predict_attractive(sentence, category):
    indexed_sentence = [AttractiveData.TEXT.vocab.stoi[t] for t in sentence]
    indexed_category = [AttractiveData.CATEGORIES_LABEL.vocab.stoi[category]]
    tensor_sentence = torch.LongTensor(indexed_sentence).to(AttractiveData.device)
    tensor_category = torch.LongTensor(indexed_category).to(AttractiveData.device)

    tensor_sentence = tensor_sentence.unsqueeze(1)
    tensor_category = tensor_category

    prediction = load_model(tensor_sentence, tensor_category)
    
    return prediction

In [14]:
# train mean = 3.2, test mean = 2.8
predict_list = []
for i, sentence in enumerate(AttractiveData.test_data):
    prediction = predict_attractive(sentence.Headline, sentence.Category)
    # predict_list.append(prediction.item() - 3.2 + 2.8)
    predict_list.append(prediction.item())
AttractiveData.df_test['Label'] = predict_list
AttractiveData.df_test[['ID', 'Label']].to_csv(config['save_name'] + '.csv', index=False)

In [15]:
# train_category = list(AttractiveData.CATEGORIES_LABEL.vocab.freqs)
# test_category = list(AttractiveData.df_test['Category'].value_counts().keys())
# for each_test in test_category:
#     if each_test not in train_category:
#         print(each_test)
# print()
# for each_train in train_category:
#     if each_train not in test_category:
#         print(each_train)

## Below just for fun guess

In [16]:
import statistics
from sklearn.metrics import mean_squared_error

In [17]:
a = AttractiveData.df_train['Label'].to_list()
statistics.mean(a), statistics.stdev(a)

(3.150408496732026, 0.729501519321601)

In [18]:
statistics.mean(predict_list), statistics.stdev(predict_list)

(2.9158543301048785, 0.35573075718956854)

In [19]:
all_28 = [2.8] * len(predict_list)
mean_squared_error(all_28, predict_list)

0.1394091332225965

In [14]:
# LSTM my best
# statistics.mean(predict_list), statistics.stdev(predict_list)

(2.8167915543795683, 0.14611407210842048)